### Lendo o arquivo - CSV

In [261]:
import zipfile
import pandas as pd

# Setando as configuracoes do pandas ao visualizar dados
pd.set_option('display.max_columns', None)

with zipfile.ZipFile("../01 - Banco de dados/netflix_titles.csv.zip") as myzip:
    with myzip.open(myzip.namelist()[0]) as mycsv:
        df = pd.read_csv(mycsv)
        df = df.convert_dtypes()


### Salvando dados no banco de dados

In [262]:
from sqlalchemy import URL, create_engine
from IPython import get_ipython
import sys

url_object = URL.create(
    "postgresql+psycopg2",
    username="analista",
    password="32TSb+@tL955",  # plain (unescaped) text
    host="localhost",
    database="netflix",
)

engine = create_engine(url_object)

try:
    with engine.begin() as conn: pass
    print("✅ Conectado com sucesso")
except:
    print("⚠️ Erro na conexao com o banco")
    if not get_ipython(): sys.exit()



✅ Conectado com sucesso


In [263]:
kwargs={
    "con":engine,
    "index":False,
    "if_exists":"replace",
    "schema":"netflix",
}



In [264]:
# Salvando base geral no banco
df.to_sql("netflix_titles", **kwargs)
df.head(5)
df.shape

(7787, 12)

In [265]:
# Verificando dados nullos
df.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [266]:
# tratando dados nulos
df.loc[ df['director'].isnull(), "director" ] = "Unknown"
df.loc[ df['cast'].isnull(), "cast" ] = "Unknown"
df.loc[ df['country'].isnull(), "country" ] = "Unknown"
df.loc[ df['rating'].isnull(), "rating" ] = "Unknown"
df.loc[ df['date_added'].isnull(), "date_added" ] = "Unknown"
df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [267]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [268]:
# Dimensao title
dim_title = df[["show_id","title", "description", "cast"]].copy()
dim_title["title_id"] = dim_title.index + 1
dim_title.to_sql("dim_title", **kwargs)

787

In [269]:
# Dimensao type
dim_type = df[['type']].drop_duplicates()
dim_type.sort_values("type",inplace=True)
dim_type.reset_index(drop="index", inplace=True)
dim_type["type_id"] = dim_type.index + 1
dim_type.to_sql("dim_type", **kwargs)

2

In [270]:
# Dimensao director
dim_director = df[['director']].drop_duplicates()
dim_director.sort_values("director",inplace=True)
dim_director.reset_index(drop="index", inplace=True)
dim_director["director_id"] = dim_director.index + 1
dim_director.to_sql("dim_director", **kwargs)


50

In [271]:
# Dimensao country
dim_country = df[['country']].drop_duplicates()
dim_country.sort_values("country",inplace=True)
dim_country.reset_index(drop="index", inplace=True)
dim_country["country_id"] = dim_country.index + 1
dim_country.to_sql("dim_country", **kwargs)


682

In [272]:
# Dimensao rating
dim_rating = df[['rating']].drop_duplicates()
dim_rating.sort_values("rating",inplace=True)
dim_rating.reset_index(drop="index", inplace=True)
dim_rating["rating_id"] = dim_rating.index + 1
dim_rating.to_sql("dim_rating", **kwargs)


15

In [273]:
# Dimensao duration
dim_duration = df[['duration']].drop_duplicates()
dim_duration.sort_values("duration",inplace=True)
dim_duration.reset_index(drop="index", inplace=True)
dim_duration["duration_id"] = dim_duration.index + 1
dim_duration.to_sql("dim_duration", **kwargs)


216

In [274]:
# Dimensao listed_in
dim_listed_in = df[['listed_in']].drop_duplicates()
dim_listed_in.sort_values("listed_in",inplace=True)
dim_listed_in.reset_index(drop="index", inplace=True)
dim_listed_in["listed_in_id"] = dim_listed_in.index + 1
dim_listed_in.to_sql("dim_listed_in", **kwargs)


492

In [275]:
# Dimensao tempo
dim_tempo = df[["show_id","date_added"]].copy()
dim_tempo["date_added_value"] = pd.to_datetime(dim_tempo["date_added"].str.strip(), errors="coerce")
dim_tempo = dim_tempo[dim_tempo["date_added_value"].notnull()]
dim_tempo["year"] = dim_tempo["date_added_value"].dt.year
dim_tempo["month"] = dim_tempo["date_added_value"].dt.month
dim_tempo["day_of_week"] = dim_tempo["date_added_value"].dt.day_of_week
dim_tempo.sort_values(by="date_added_value", inplace=True)
dim_tempo.reset_index(inplace=True, drop="index")
dim_tempo["tempo_id"] = dim_tempo.index + 1
dim_tempo.to_sql("dim_tempo", **kwargs)


777

In [276]:
dim_tempo.shape

(7777, 7)

In [277]:
# Fato TITLE

columns_fato = [
    "title_id",
    "type_id",
    "director_id",
    "country_id",
    "rating_id",
    "duration_id",
    "listed_in_id",
    "tempo_id",
    "release_year",
]
df_fim = df.copy()

df_fim = df_fim.merge(dim_title, how="left", on="show_id")
df_fim = df_fim.merge(dim_type, how="left", on="type")
df_fim = df_fim.merge(dim_director, how="left", on="director")
df_fim = df_fim.merge(dim_country, how="left", on="country")
df_fim = df_fim.merge(dim_rating, how="left", on="rating")
df_fim = df_fim.merge(dim_duration, how="left", on="duration")
df_fim = df_fim.merge(dim_listed_in, how="left", on="listed_in")
df_fim = df_fim.merge(dim_tempo, how="left", on="show_id")

fato_title = df_fim[columns_fato]
fato_title.to_sql("fato_title", **kwargs)

fato_title.shape, df.shape

((7787, 9), (7787, 12))

In [280]:
fato_title.head(15)

,title_id,type_id,director_id,country_id,rating_id,duration_id,listed_in_id,tempo_id,release_year
0,1,2,3847,40,9,148,399,6906.0,2020
1,2,1,1841,309,9,210,311,575.0,2016
2,3,1,1290,380,6,191,337,3422.0,2011
3,4,1,3446,550,5,195,48,1667.0,2009
4,5,1,3177,550,5,31,295,5705.0,2008
5,6,2,3419,449,9,1,398,1201.0,2016
6,7,1,3991,138,9,212,337,6516.0,2019
7,8,1,2025,550,6,25,295,5234.0,1997
8,9,1,3486,230,7,24,337,4022.0,2019
9,10,1,3893,230,9,54,340,1742.0,2008
